### Set up

In [1]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

pg_store = Neo4jPropertyGraphStore(
    username='neo4j',
    password='llamaindex',
    url="bolt://localhost:7688",
)

### Inserting:
- Now that we have a store initialized, we can put something in its
- Inserting into a property graph store consist of inserting nodes
    - EntityNode: containing some labeled person, place or thing
    - ChunkNode: containing some source text that an entity or relation came from
- And inserting `Relation`s (linking multiple nodes) 

In [2]:
from llama_index.core.graph_stores.types import EntityNode, ChunkNode, Relation

entity1 = EntityNode(label="PERSON", name='Logan', properties={'age':28})
entity2 = EntityNode(label="ORGANIZATION", name='LlamaIndex')

relation = Relation(
    label='WORKS_FOR',
    source_id=entity1.id,
    target_id=entity2.id,
    properties={"since": 2023}
)

With some entities and relation defined, we can insert them

In [3]:
pg_store.upsert_nodes([entity1, entity2])
pg_store.upsert_relations([relation])

And if you wanted, we could also define a text chunk that these came from

In [5]:
from llama_index.core.schema import TextNode

source_node = TextNode(text='Logan (age 28), works for LlamaIndex since 2023')
relations = [
    Relation(
        label='MENTIONS',
        source_id=source_node.node_id,
        target_id=entity1.id
    ),
    Relation(
        label='MENTIONS',
        source_id=source_node.node_id,
        target_id=entity2.id
    )
]

pg_store.upsert_llama_nodes([source_node])
pg_store.upsert_relations(relations)

### Retrieving:

Now that our graph is populated with some nodes and relations, we can access some of the retrieval function!

In [6]:
# get a node
kg_nodes = pg_store.get(ids=[entity1.id])
print(kg_nodes)


[EntityNode(label='__Node__', embedding=None, properties={'age': 28, 'name': 'Logan'}, name='Logan')]


In [7]:
# get using properties
kg_nodes = pg_store.get(properties={'age': 28})
print(kg_nodes)

[EntityNode(label='__Node__', embedding=None, properties={'age': 28, 'name': 'Logan'}, name='Logan')]


In [8]:
# get paths from a node
paths = pg_store.get_rel_map(kg_nodes, depth=1)
for path in paths:
    print(f'{path[0].id} -> {path[1].id} -> {path[2].id}')

Logan -> WORKS_FOR -> LlamaIndex


In [9]:
# Run a cypher query (this will get all entity nodes)
query = "match (n: `__Entity__`) return n"
result = pg_store.structured_query(query)
print(result)

[{'n': {'name': 'Logan', 'id': 'Logan', 'age': 28}}, {'n': {'name': 'LlamaIndex', 'id': 'LlamaIndex'}}]


In [11]:
# get the original text node back
llama_nodes = pg_store.get_llama_nodes([source_node.node_id])
print(llama_nodes[0].text)

Logan (age 28), works for LlamaIndex since 2023


### Upserting

You may have notices that all the insert operations are actually upserts! As long as the ID of the node is the same, we can avoid duplicating data \
Lets update a node

In [12]:
new_node = EntityNode(
    label='PERSON', name='Logan', properties={"age":28, "location": 'Canada'}
)
pg_store.upsert_nodes([new_node])

In [13]:
nodes = pg_store.get(properties={"age": 28})
print(nodes)

[EntityNode(label='__Node__', embedding=None, properties={'location': 'Canada', 'age': 28, 'name': 'Logan'}, name='Logan')]


### Deleting:
Deletion works similar to get(), with both IDS and properties \
Lets clean up our graph for a fresh start

In [14]:
# delete our entities
pg_store.delete(ids=[entity1.id, entity2.id])

# delete our text nodes
pg_store.delete([source_node.node_id])


In [15]:
# delete our entities
pg_store.delete(ids=[entity1.id, entity2.id])

# delete our text nodes
pg_store.delete([source_node.node_id])